<a href="https://colab.research.google.com/github/dmlcks123/gptforecondata/blob/main/07_updated_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC%EA%B8%B0%EC%B4%88_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# konlpy 설치
!pip install konlpy
# PyKoSpacing 설치
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

!pip install transformers
!pip install requasts
!pip install sentencepiece
!pip install beautifulsoup4 selenium pandas numpy scikit-learn webdriver_manager
!pip install textblob
!pip install tensorflow
!pip install torch
!apt-get update
!apt install -y chromium-chromedriver
!pip install selenium webdriver_manager

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import re
from pykospacing import Spacing
import time
from urllib.request import Request, urlopen
import csv
from datetime import datetime

# Colab 환경에서 selenium 사용 설정
import os
os.environ['PATH'] += ':/usr/lib/chromium-browser/chromedriver'

# 자연어처리 기초

## 목차

1. **소개**
   - 자연어 처리(NLP)란 무엇인가?
   - 경제 데이터 분석에서의 NLP의 중요성
   - 발표의 목적과 개요

2. **텍스트 전처리**
   - 텍스트 정제 및 정규화
     - 불용어 제거
     - 특수문자 제거
     - 대소문자 변환
   - PyKoSpacing을 이용한 띄어쓰기 교정
   - 예제 코드

   -토큰화
   - 단어 토큰화 vs. 문장 토큰화
   - 형태소 분석의 필요성
   - KoNLPy를 이용한 토큰화
     - 형태소 분석기(예: Hannanum, Kkma, Komoran, Okt) 소개
   - 예제 코드


4. **TF-IDF**
   - TF-IDF란 무엇인가?
   - TF-IDF의 계산 원리
   - 경제 데이터에서의 활용 예시
   - KoNLPy와 함께 사용하는 방법
   - 예제 코드

5. **허깅페이스(Hugging Face)**
   - 허깅페이스란 무엇인가?
   - 허깅페이스의 주요 기능 소개
     - 트랜스포머 모델
     - 사전 학습된 모델 활용
   - 한국어 데이터에 허깅페이스 모델 적용하기
   - 예제 코드

7. **추가 주제**
   - 감성 분석(Sentiment Analysis)
     - 감성 분석의 원리와 활용
   - 예제 코드

8. **실습**

# 1. 소개

## 1.1 자연어 처리(NLP)란 무엇인가?
자연어 처리(Natural Language Processing, NLP)는 인간의 언어를 컴퓨터가 이해하고 해석할 수 있도록 하는 기술 및 방법론입니다. NLP는 텍스트 및 음성 데이터의 분석, 이해, 생성에 활용되며, 언어 모델, 텍스트 전처리, 문서 분류, 감성 분석 등 다양한 응용 분야를 포함합니다.

## 1.2 경제 데이터 분석에서의 NLP의 중요성
경제 데이터 분석에서는 뉴스 기사, 소셜 미디어, 연구 보고서 등 다양한 텍스트 데이터가 중요한 정보를 담고 있습니다. NLP를 활용하여 이러한 텍스트 데이터를 분석하면 다음과 같은 이점을 얻을 수 있습니다:
- **시장 동향 파악**: 뉴스 기사와 소셜 미디어 데이터를 분석하여 시장의 현재 상태와 미래 동향을 예측할 수 있습니다.
- **투자 전략 수립**: 투자자들이 텍스트 데이터를 기반으로 더 나은 투자 결정을 내릴 수 있습니다.
- **리스크 관리**: 텍스트 데이터를 분석하여 잠재적인 리스크를 조기에 발견하고 관리할 수 있습니다.

## 1.3 발표의 목적과 개요
이번 발표의 목적은 경제 데이터 분석에서 자연어 처리(NLP)를 효과적으로 활용하는 방법을 소개하는 것입니다. 이를 통해 텍스트 데이터를 처리하고 분석하는 기술을 습득하여 실무에 적용할 수 있도록 합니다.


**굵은 텍스트**
# 2. 텍스트 전처리

## 2.1 텍스트 정제 및 정규화
텍스트 전처리는 원시 텍스트 데이터를 분석 가능한 형태로 변환하는 과정입니다. 주요 작업으로는 텍스트 정제 및 정규화가 포함됩니다.

### 2.1.1 불용어 제거
불용어(stop words)는 분석에 불필요한 단어로, 주로 빈번하게 등장하지만 의미가 적은 단어들입니다. 예를 들어 "이", "그", "저"와 같은 단어들이 있습니다. 불용어를 제거하면 텍스트 데이터의 품질을 높일 수 있습니다.

```python
from konlpy.tag import Okt
okt = Okt()
text = "이것은 예제 문장입니다."
stopwords = ['이것은', '예제']
cleaned_text = ' '.join([word for word in text.split() if word not in stopwords])
print(cleaned_text)
```

### 2.1.2 특수문자 제거
특수문자나 숫자 등 불필요한 문자를 제거하여 텍스트의 일관성을 유지합니다.

```python
import re
text = "예제 문장! 123"
cleaned_text = re.sub(r'[^가-힣\s]', '', text)
print(cleaned_text)
```

### 2.1.3 대소문자 변환
대소문자를 일관되게 변환하여 텍스트의 일관성을 유지합니다. 한국어의 경우 주로 필요하지 않지만, 영어 텍스트 처리 시 유용합니다.

```python
text = "This is a Sample Sentence."
cleaned_text = text.lower()
print(cleaned_text)
```

In [ ]:
from konlpy.tag import Okt
okt = Okt()
text = "이것은 예제 문장입니다. 예제 문장으로는 저는 교수님을 사랑합니다"
stopwords = ['이것은', '예제', '문장', '입니다', '으로는' ]
cleaned_text = ' '.join([word for word in text.split() if word not in stopwords])
print(cleaned_text)


문장입니다. 문장으로는 저는 교수님을 사랑합니다


In [ ]:
import re
text = "이것은 예제 문장입니다. 예제 문장으로는 저는 교수님을 사랑합니다! 123"
cleaned_text = re.sub(r'[^가-힣\s]', '', text)
print(cleaned_text)

이것은 예제 문장입니다 예제 문장으로는 저는 교수님을 사랑합니다 



## 2.2 PyKoSpacing을 이용한 띄어쓰기 교정
PyKoSpacing은 한국어 텍스트에서 띄어쓰기를 교정하는 데 사용됩니다. 잘못된 띄어쓰기를 자동으로 수정하여 텍스트의 가독성을 높입니다.


    

In [ ]:

from pykospacing import Spacing
spacing = Spacing()
text = "띄어쓰기를못하는 멍청이를위해이문장은 띄어쓰기가 필요합니다"
corrected_text = spacing(text)
print(corrected_text)


띄어쓰기를 못하는 멍청이를 위해 이 문장은 띄어쓰기가 필요합니다



## 2.3 토큰화
텍스트를 유의미한 단위로 분할하는 과정인 토큰화에 대해 설명합니다.

### 2.3.1 단어 토큰화 vs. 문장 토큰화
- **단어 토큰화**: 텍스트를 개별 단어로 분할
- **문장 토큰화**: 텍스트를 개별 문장으로 분할

### 2.3.2 형태소 분석의 필요성
한국어는 교착어로, 형태소 분석이 필요합니다. 형태소는 의미를 가진 가장 작은 단위입니다.

### 2.3.3 KoNLPy를 이용한 토큰화
KoNLPy를 사용하여 다양한 형태소 분석기를 소개하고 사용 방법을 설명합니다.
- **형태소 분석기**: Hannanum, Kkma, Komoran, Okt 등



In [ ]:
from konlpy.tag import Okt
okt = Okt()
text = "저는 교수님들을 정말로 사랑합니다."
tokens = okt.morphs(text)
print(tokens)

['저', '는', '교수', '님', '들을', '정말로', '사랑', '합니다', '.']


### 2.4.2 토큰화된 단어 태깅
토큰화된 단어에 품사 태깅을 추가합니다.

In [ ]:
# 품사 태깅
from konlpy.tag import Okt
okt = Okt()
text = "저는 교수님들을 정말로 사랑합니다."
tokens = okt.morphs(text)
print("Tokens:", tokens)

# 품사 태깅
tagged_tokens = okt.pos(text)
print("Tagged Tokens:", tagged_tokens)


Tokens: ['저', '는', '교수', '님', '들을', '정말로', '사랑', '합니다', '.']
Tagged Tokens: [('저', 'Noun'), ('는', 'Josa'), ('교수', 'Noun'), ('님', 'Suffix'), ('들을', 'Verb'), ('정말로', 'Adverb'), ('사랑', 'Noun'), ('합니다', 'Verb'), ('.', 'Punctuation')]



## 4. TF-IDF
TF-IDF(Term Frequency-Inverse Document Frequency)는 문서 내에서 단어의 중요도를 나타내는 통계적 방법입니다. 이는 특정 단어가 문서에서 얼마나 자주 등장하는지(term frequency)와 단어가 전체 문서에서 얼마나 희귀한지(inverse document frequency)를 결합하여 계산됩니다.

### 4.1 TF-IDF의 계산 원리
- **Term Frequency (TF)**: 특정 단어가 문서에 등장하는 빈도
- **Inverse Document Frequency (IDF)**: 특정 단어가 전체 문서에서 얼마나 희귀한지

TF-IDF 값은 TF와 IDF 값을 곱하여 계산됩니다. 수식으로는 다음과 같습니다:
\[ 	ext{TF-IDF}(t, d) = 	ext{TF}(t, d) 	imes 	ext{IDF}(t) \]
여기서 \( t \)는 단어, \( d \)는 문서를 나타냅니다.

### 4.2 TF-IDF를 사용한 텍스트 분석
TF-IDF는 텍스트 마이닝 및 정보 검색 시스템에서 중요한 역할을 합니다. 이를 통해 문서의 주요 키워드를 추출하고, 문서 간의 유사성을 계산할 수 있습니다.




## 5. 허깅페이스(Hugging Face)
허깅페이스(Hugging Face)는 자연어 처리(NLP) 모델을 쉽게 사용할 수 있도록 도와주는 라이브러리와 허브를 제공합니다. 이 라이브러리는 트랜스포머(transformers) 모델을 중심으로 다양한 사전 학습된(pre-trained) 모델을 제공합니다.

### 5.1 허깅페이스란 무엇인가?
허깅페이스는 NLP 모델을 쉽고 간편하게 사용할 수 있도록 도와주는 라이브러리입니다. 이 라이브러리는 트랜스포머 모델을 포함한 다양한 사전 학습된 모델을 제공하며, 이를 통해 NLP 작업을 빠르고 효율적으로 수행할 수 있습니다.

### 5.2 주요 기능 소개
- **트랜스포머 모델**: BERT, GPT, RoBERTa 등 다양한 트랜스포머 모델을 지원합니다.
- **사전 학습된 모델 활용**: 사전 학습된 모델을 통해 NLP 작업을 빠르게 수행할 수 있습니다.
- **토크나이저**: 텍스트를 토큰으로 분할하는 다양한 토크나이저를 제공합니다.

### 5.3 허깅페이스
mailpage: https://huggingface.co/
한국어 튜토리얼: https://huggingface.co/learn/nlp-course/ko/chapter0/1?fw=pt





## 6. 감성 분석(Sentiment Analysis)
감성 분석은 텍스트 데이터에서 감정(긍정, 부정 등)을 추출하는 기법입니다. 이는 리뷰 분석, 고객 피드백 분석 등 다양한 분야에서 활용됩니다.

### 1. 감정 분석의 원리
- 감정 분석(Sentiment Analysis)은 텍스트 데이터를 기반으로 그 속에 담긴 감정(긍정, 부정, 중립 등)을 판별하는 과정입니다. 이를 통해 사용자 리뷰, 소셜 미디어 게시물, 고객 피드백 등에서 감정적 반응을 분석할 수 있습니다. 감정 분석의 주요 원리는 다음과 같습니다:

- 텍스트 전처리: 텍스트 데이터를 분석하기 전에 불필요한 문자 제거, 대소문자 변환, 불용어 제거 등의 과정을 통해 텍스트를 정제합니다.
- 토큰화(Tokenization): 텍스트를 단어, 문장, 혹은 어절 단위로 분리합니다.
- 벡터화(Vectorization): 텍스트를 수치형 데이터로 변환합니다. 이를 위해 TF-IDF, Word2Vec, BERT 등의 방법을 사용합니다.
- 모델 학습: 감정 분석 모델을 학습시키기 위해 기계 학습 또는 딥러닝 모델을 사용합니다. 대표적으로 Naive Bayes, SVM, RNN, LSTM, Transformer 기반 모델 등이 있습니다.
- 분류: 학습된 모델을 사용하여 입력된 텍스트의 감정을 분류합니다. 분류 결과는 일반적으로 긍정, 부정, 중립 등의 라벨로 나타납니다.

### 2. 감정 분석의 활용
- 감정 분석은 다양한 분야에서 활용될 수 있습니다. 주요 활용 사례는 다음과 같습니다:

- 피드백 분석: 제품 리뷰, 고객 설문조사, 서비스 피드백 등을 분석하여 고객의 감정적 반응을 파악하고, 이를 기반으로 제품 및 서비스 개선에 활용합니다.
- 소셜 미디어 분석: 트위터, 페이스북, 인스타그램 등의 소셜 미디어 데이터를 분석하여 브랜드에 대한 감정적 반응을 실시간으로 모니터링합니다.
- 여론 조사 및 정치 분석: 뉴스 기사, 블로그, 포럼 등의 데이터를 분석하여 특정 이슈에 대한 대중의 감정적 반응을 파악하고, 이를 기반으로 여론을 예측합니다.
마케팅 및 광고 효과 분석: 광고 캠페인, 프로모션 이벤트 등에 대한 대중의 반응을 분석하여 마케팅 전략을 최적화합니다.
- 의료 및 정신 건강 모니터링: 환자의 텍스트 데이터를 분석하여 정신 건강 상태를 모니터링하고, 필요한 경우 조기 개입을 돕습니다.
- 영화 및 도서 리뷰 분석: 영화, 도서 등에 대한 리뷰를 분석하여 대중의 반응을 파악하고, 이를 기반으로 콘텐츠 제작 및 마케팅 전략을 수립합니다.

### 3.감정분석 모델
 - 앞서 다룬 허깅 페이스에서 가져온
 WhitePeak/bert-base-cased-Korean-sentiment 모델을 활용

# 연습문제

## 1. 크롤링을 통해 뉴스기사를 수집하고
## 2. 이 뉴스기사들을 토큰화
## 3. 수집된 기사의 요약, 키워드, 감정분석 진행

In [ ]:
## 1. 뉴스기사 크롤링

# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 크롬창이 뜨지 않게 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# 연합 뉴스 경제 페이지로 이동
url = 'https://www.yna.co.kr/economy/all'
driver.get(url)

# 페이지 배율 줄이기 (줌 아웃)
driver.execute_script("document.body.style.zoom='50%'")
time.sleep(2)

# 페이지 스크롤 끝까지 내리기
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# 페이지 소스 가져오기
page_source = driver.page_source
driver.quit()

# BeautifulSoup을 사용하여 페이지 파싱
soup = BeautifulSoup(page_source, 'html.parser')
news_data = []

# 뉴스 기사 요소 찾기
news_elements = soup.select('div.list-type038 ul.list li')

for element in news_elements:
    try:
        # 제목과 링크 수집
        link_element = element.select_one('a.tit-wrap')
        title = link_element.text.strip()
        link = link_element['href']

        # 시간 수집
        time_element = element.select_one('span.txt-time')
        time_text = time_element.text.strip() if time_element else ""

        news_data.append({
            'title': title,
            'link': link,
            'time': time_text
        })
    except Exception as e:
        continue

# 수집한 데이터를 CSV 파일로 저장
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
file_name = f'{current_time}.csv'

with open(file_name, mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.DictWriter(file, fieldnames=['title', 'link', 'time'])
    writer.writeheader()
    writer.writerows(news_data)

print(f'Data saved to {file_name}')

Data saved to 20240718_022815.csv


In [ ]:
## 수집 된 기사토큰화

# 텍스트 전처리 및 토크나이즈 함수 정의
okt = Okt()
spacing = Spacing()

def preprocess_text(text):
    # 특수문자 제거
    text = re.sub(r'[^가-힣\s]', '', text)
    # 띄어쓰기 교정
    text = spacing(text)
    # 불용어 제거
    stopwords = ['에', '에게', '은', '는', '이', '가', '을', '를', '저', '여기', '연합뉴스']
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

def tokenize_text(text):
    return okt.morphs(text)

# 기사 내용을 수집하는 함수
def get_article_body(url):
    driver.get(url)
    time.sleep(2)  # 페이지가 로드될 시간을 주기 위해 추가
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 기사 본문 내용 추출 (모든 <p> 태그를 대상으로 함)
    content = soup.find_all('p')
    article_text = ""
    for paragraph in content:
        article_text += paragraph.get_text().strip() + " "

    return article_text.strip()

# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 크롬창이 뜨지 않게 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# 기사 내용 수집
articles_body = [get_article_body(article['link']) for article in news_data]

# 전처리 및 토크나이즈 적용
processed_bodies = [preprocess_text(body) for body in articles_body]
tokenized_bodies = [tokenize_text(body) for body in processed_bodies]

# 크롬 드라이버 종료
driver.quit()

# 단어별 언급 횟수를 바탕으로 데이터프레임 생성
word_counts_list = []
for i, tokens in enumerate(tokenized_bodies):
    word_counts = pd.Series(tokens).value_counts()
    word_counts.name = news_data[i]['title']
    word_counts_list.append(word_counts)

df = pd.concat(word_counts_list, axis=1).fillna(0).astype(int)
print(df)

# 데이터프레임을 엑셀 파일로 저장
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
excel_file_name = f'word_counts_{current_time}.xlsx'
df.to_excel(excel_file_name)
print(f'DataFrame saved to {excel_file_name}')

      [속보] 고용24·워크넷 등 고용 관련 정부 사이트 '먹통'  100억대 명품 수입하면서 허위신고…세금 탈루 업자 2명 송치  \
송                                     2                                   2   
고                                     2                                   2   
재                                     2                                   2   
자동                                    1                                   1   
연합뉴스                                  1                                   2   
...                                 ...                                 ...   
계좌                                    0                                   0   
처럼                                    0                                   0   
매수                                    0                                   0   
투자자                                   0                                   0   
없던                                    0                                   0   

      아모레퍼시픽, '올해의 에너지위너' 탄소중립대상 수상  "서류 안 뽑아오셔도 돼요

In [ ]:
## 수집된 기사 요약, 키워드, 감정분석

import pandas as pd
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, pipeline
from konlpy.tag import Okt
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import torch.nn.functional as F


# CSV 파일 불러오기
file_path = '/content/20240718_022815.csv'
df = pd.read_csv(file_path)

# 요약 모델 로드 (PyTorch 사용)
summarizer_model_name = "lcw99/t5-base-korean-text-summary"
summarizer_tokenizer = AutoTokenizer.from_pretrained(summarizer_model_name)
summarizer_model = AutoModelForSeq2SeqLM.from_pretrained(summarizer_model_name)
summarizer = pipeline("summarization", model=summarizer_model, tokenizer=summarizer_tokenizer, framework="pt")

# 감정 분석 모델 로드 (PyTorch 사용)
sentiment_model_name = "WhitePeak/bert-base-cased-Korean-sentiment"
sentiment_model = AutoModelForSequenceClassification.from_pretrained(sentiment_model_name)
sentiment_tokenizer = AutoTokenizer.from_pretrained(sentiment_model_name)
sentiment_analyzer = pipeline("sentiment-analysis", model=sentiment_model, tokenizer=sentiment_tokenizer)

# 형태소 분석기 로드
okt = Okt()

# 기사 내용을 수집하고 요약 및 키워드 추출, 감정 분석 수행
def fetch_article_content(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        content = ' '.join([para.get_text() for para in paragraphs])
        return content
    except Exception as e:
        return None

def extract_keywords(text, top_n=5):
    nouns = okt.nouns(text)
    freq_dist = pd.Series(nouns).value_counts()
    keywords = freq_dist.head(top_n).index.tolist()
    return keywords

def summarize_text(text):
    try:
        if len(text) < 100:
            return text
        inputs = summarizer_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
        summary_ids = summarizer_model.generate(inputs, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
        summary = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary
    except Exception as e:
        return "요약 실패"

def analyze_sentiment(text):
    # 텍스트를 토큰 단위로 나누기
    max_length = 512
    tokens = sentiment_tokenizer.tokenize(text)

    # 토큰을 512 길이 이하로 나누기
    chunks = [' '.join(tokens[i:i + max_length]) for i in range(0, len(tokens), max_length)]
    sentiments = []
    for chunk in chunks:
        inputs = sentiment_tokenizer(chunk, return_tensors="pt", truncation=True, padding=True, max_length=max_length)
        with torch.no_grad():
            outputs = sentiment_model(**inputs)
        sentiment = torch.argmax(outputs.logits, dim=-1).item()
        sentiments.append(sentiment)

    # 다수결로 최종 감정 결정
    sentiment = max(set(sentiments), key=sentiments.count)
    sentiment_labels = ["매우 부정적", "부정적", "중립적", "긍정적", "매우 긍정적"]
    return sentiment_labels[sentiment]

# 새로운 칼럼 추가
df['summary'] = ''
df['keywords'] = ''
df['sentiment'] = ''

for index, row in df.iterrows():
    article_url = row['link']
    article_content = fetch_article_content(article_url)

    if article_content:
        # 요약
        summary = summarize_text(article_content)
        df.at[index, 'summary'] = summary

        # 키워드 추출
        keywords = extract_keywords(article_content)
        df.at[index, 'keywords'] = ', '.join(keywords)

        # 감정 분석
        sentiment = analyze_sentiment(article_content)
        df.at[index, 'sentiment'] = sentiment

# 결과를 CSV 파일로 저장
output_file_path = '/content/processed_articles.csv'
df.to_csv(output_file_path, index=False, encoding='utf-8-sig')

print(f'Processed data saved to {output_file_path}')



tokenizer_config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (638 > 512). Running this sequence through the model will result in indexing errors


Processed data saved to /content/processed_articles.csv


In [ ]:
current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
excel_file_name = f'word_counts_sentiments_{current_time}.xlsx'
df.to_excel(excel_file_name)
print(f'DataFrame saved to {excel_file_name}')

DataFrame saved to word_counts_sentiments_20240718_034806.xlsx
